# 4. Pose Registration

In the following notebook we use 2 pose registration algorithms. RANSAC and ICP in order to be able to track the pose of the scanned bell pepper using the 3D model. 

## 1. Importing Libraries

In [1]:
import open3d as o3d
import numpy as np
import copy
import time
import os 
import sys
import cv2
import json

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## 2. Importing Files

In [2]:
source_path = "C:\\Users\\gusta\\Desktop\\ITESM_Desktop\\maestria\\tesis\\TercerSemestre\\realTimeICP\\refined_one_img_process\\results\\scaled_aligned_model\\pancake_right_inclination.ply"
target_path = "C:\\Users\\gusta\\Desktop\\ITESM_Desktop\\maestria\\tesis\\TercerSemestre\\realTimeICP\\refined_one_img_process\\results\\translated_kinect_scan\\translated_kinect_scan.ply"

Helper function for visualization

In [3]:
# scaled_pcd2 is the rightly scaled model point cloud
# target is the kinect scan
def draw_registration_result(source, target, transformation):
  source_temp = copy.deepcopy(source)
  target_temp = copy.deepcopy(target)
  source_temp.paint_uniform_color([1, 0.706, 0])
  target_temp.paint_uniform_color([0, 0.651, 0.929])
  source_temp.transform(transformation)
  o3d.visualization.draw_geometries([source_temp, target_temp])
  # Note: Since the functions "transform" and "paint_uniform_color" change the point cloud,
  # we call copy.deep to make copies and protect the original point clouds.

## 3. Global registration 

### Global Registration Initialization
#### Extract Geometric Features

We downsample the pointcloud to estimate normals, then compute the FPFH feature for each point. The FPFH feature is a 33-dimensional vector that describes the local geometric property of a point. A nearest neighbor query in the 33-dimensional space can return points with similar local geometric structures. 

In [4]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size of %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with  search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30)
    )
    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down, 
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100)
    )
    return pcd_down, pcd_fpfh

#### Input
The code below reads a source point cloud and a target point cloud from two files, they are misaligned with an identity matrix as transformation.

In [5]:
def prepare_dataset(voxel_size):
    print(":: Load two point clouds and disturb the initial pose.")
    source = o3d.io.read_point_cloud(source_path)
    target = o3d.io.read_point_cloud(target_path)
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], 
                             [1.0, 0.0, 0.0, 0.0], 
                             [0.0, 1.0, 0.0, 0.0], 
                             [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

In [6]:
voxel_size = 0.005 # means 5cm for the original dataset... gotta check on mine
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(
    voxel_size=voxel_size)

:: Load two point clouds and disturb the initial pose.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Controlador no válido. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Controlador no válido. 
:: Downsample with a voxel size of 0.005.
:: Estimate normal with  search radius 0.010.
:: Compute FPFH feature with search radius 0.025.
:: Downsample with a voxel size of 0.005.
:: Estimate normal with  search radius 0.010.
:: Compute FPFH feature with search radius 0.025.


### RANSAC
RANSAC is utilized for global registration. In each RANSAC iteration, ransac_n random points are picked from the source point cloud. Their corresponding points in the target point cloud are detected by querying the nearest neighbor in the 33-dimensional FPFH feature space. A pruning step takes fast pruning algorithms to quickly reject false matches early. 

Open3D provides the following prunning algorithms: 
* CorrespondenceCheckerBasedOnDistance checks if aligned point clouds are close (less than the specified threshold). 
* CorrespondenceCheckBasedOnEdgeLength checks if the lengths of any two arbitrary edges (line formed by two vertices) individually drawn from source and target correspondences are similar. This tutorial checks that ||edge_source|| > 0.9 * || edge_target || and ||edge_target|| > 0.9 * ||edge_source|| are true. 
* CorrespondeceCheckerBasedOnNormal considers vertex normal affinity of any correspondences. It computes the dot product of two normal vectors. It takes a radian value for the threshold. 

Only matches that pass through the prunning step are used to compute a transformation, which is validated on the entire point cloud. The core function is registration_ransac_based_on_feature_matching. The most important hyperparameter of this function is RANSACConvergenceCriteria. It defines the maximum number of RANSAC iterations and the confidence probability. The larger these 2 numbers are, the more accurate the result is, but also the more time the algorithm takes. 

In [7]:
def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds. ")
    print("   Since the downsampling voxel sixe is %.3f, " % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True, 
        distance_threshold, 
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False), 3, 
        [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9
            ),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold
            )
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999)
    )
    return result

In [8]:
result_ransac = execute_global_registration(source_down, target_down, 
                                            source_fpfh, target_fpfh, 
                                            voxel_size)
print(result_ransac)
draw_registration_result(source_down, target_down, result_ransac.transformation)

:: RANSAC registration on downsampled point clouds. 
   Since the downsampling voxel sixe is 0.005, 
   we use a liberal distance threshold 0.007.
RegistrationResult with fitness=4.210526e-01, inlier_rmse=4.232025e-03, and correspondence_set size of 640
Access transformation to get result.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: La operación de transformación solicitada no es compatible. 


### Local refinement

For performance reasons, the registration is only performed on a heavily down-sampled point cloud. The result is not tight. We can now use Point-to-plane ICP to further refine the alignment. 

In [9]:
def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, result_ransac.transformation, 
        o3d.pipelines.registration.TransformationEstimationPointToPlane()
    )
    return result


In [10]:
def compute_normals(pcd, radius):
    pcd.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=30)
    )

In [11]:
radius_normal = voxel_size * 2
compute_normals(source, radius_normal)
compute_normals(target, radius_normal)

In [12]:
result_icp = refine_registration(source, target, source_fpfh, target_fpfh, voxel_size)
print(result_icp)
draw_registration_result(source, target, result_icp.transformation)

:: Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. This time we use a strict
   distance threshold 0.002.
RegistrationResult with fitness=2.011000e-01, inlier_rmse=1.160255e-03, and correspondence_set size of 2011
Access transformation to get result.
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


In [27]:
icp_result_path = "C:\\Users\\gusta\\Desktop\\ITESM_Desktop\\maestria\\tesis\\TercerSemestre\\realTimeICP\\refined_one_img_process\\results\\ICP_result\\icp_result_pancake.ply"
source_transformed = source.transform(result_icp.transformation)
#o3d.io.write_point_cloud(icp_result_path, source_transformed)

True

Once you've done this, you've successfully been able to estimate the pose of the scan using your model. 